In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np
import os
import time
import csv

In [2]:
path_to_file = '../shepherd/names.csv'
# Read, then decode for py2 compat.
lines = [line[0:-1].decode('utf-8') for line in open(path_to_file, 'rb').readlines()]
# length of text is the number of characters in it
print ('Number of lines: {} '.format(len(lines)))

Number of lines: 20466 


In [3]:
# Print first few card names
print(lines[:10])

['Rumors of My Death', '1996 World Champion', 'A Display of My Dark Power', 'A Reckoning Approaches', 'Abandon Hope', 'Abandon Reason', 'Abandoned Outpost', 'Abandoned Sarcophagus', 'Abattoir Ghoul', 'Abbey Gargoyles']


In [4]:
# The maximum length sentence we want for a single input in characters
seq_length = len(max(lines, key=len))
examples_per_epoch = len(lines)
seq_length

64

In [5]:
# Get the vocabular of the text
vocab = sorted(set(''.join(lines)+"#"))
print ('{} unique characters'.format(len(vocab)))

79 unique characters


In [6]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def to_array(line):
    a = np.array([char2idx[c] for c in line])
    z = np.full(seq_length - len(a), char2idx['#'],dtype=a.dtype)
    return np.append(a,z)

lines_as_int = np.array([to_array(s) for s in lines])
lines_as_int.shape

(20466, 64)

In [7]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  ' ' :   0,
  '!' :   1,
  '#' :   2,
  '&' :   3,
  "'" :   4,
  '(' :   5,
  ')' :   6,
  ',' :   7,
  '-' :   8,
  '.' :   9,
  '/' :  10,
  '0' :  11,
  '1' :  12,
  '4' :  13,
  '6' :  14,
  '9' :  15,
  ':' :  16,
  '?' :  17,
  'A' :  18,
  'B' :  19,
  ...
}


In [8]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(lines[0][0:13]), lines_as_int[0, 0:13]))

'Rumors of My ' ---- characters mapped to int ---- > [35 64 56 58 61 62  0 58 49  0 30 68  0]


In [9]:


# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(lines_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

['R' 'u' 'm' 'o' 'r' 's' ' ' 'o' 'f' ' ' 'M' 'y' ' ' 'D' 'e' 'a' 't' 'h'
 '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#'
 '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#'
 '#' '#' '#' '#' '#' '#' '#' '#' '#' '#']
['1' '9' '9' '6' ' ' 'W' 'o' 'r' 'l' 'd' ' ' 'C' 'h' 'a' 'm' 'p' 'i' 'o'
 'n' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#'
 '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#'
 '#' '#' '#' '#' '#' '#' '#' '#' '#' '#']
['A' ' ' 'D' 'i' 's' 'p' 'l' 'a' 'y' ' ' 'o' 'f' ' ' 'M' 'y' ' ' 'D' 'a'
 'r' 'k' ' ' 'P' 'o' 'w' 'e' 'r' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#'
 '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#'
 '#' '#' '#' '#' '#' '#' '#' '#' '#' '#']
['A' ' ' 'R' 'e' 'c' 'k' 'o' 'n' 'i' 'n' 'g' ' ' 'A' 'p' 'p' 'r' 'o' 'a'
 'c' 'h' 'e' 's' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#'
 '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#

In [10]:
#sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
sequences = char_dataset
#for item in sequences.take(5):
#  print(repr(''.join(idx2char[item.numpy()])))

In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Rumors of My Death#############################################'
Target data: 'umors of My Death##############################################'


In [13]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 35 ('R')
  expected output: 64 ('u')
Step    1
  input: 64 ('u')
  expected output: 56 ('m')
Step    2
  input: 56 ('m')
  expected output: 58 ('o')
Step    3
  input: 58 ('o')
  expected output: 61 ('r')
Step    4
  input: 61 ('r')
  expected output: 62 ('s')


In [14]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 63), (64, 63)), types: (tf.int64, tf.int64)>

In [15]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 24

# Number of RNN units
rnn_units = 1024

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=False,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [17]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 63, 79) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 24)            1896      
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3225600   
_________________________________________________________________
dense (Dense)                (64, None, 79)            80975     
Total params: 3,308,471
Trainable params: 3,308,471
Non-trainable params: 0
_________________________________________________________________


In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [21]:
sampled_indices

array([28, 52, 58, 73, 57, 41, 32, 45, 47, 15, 31, 74,  7, 78, 24, 65, 21,
       58, 31, 71,  1, 74,  4, 39, 32, 39, 57, 73, 36, 23,  3, 38, 43, 63,
       11, 20, 27, 67, 33,  0, 58, 41, 78, 12,  4,  8,  7, 18,  4, 31, 18,
       32, 26, 69, 60, 19,  4, 75, 39, 74, 39, 75, 58])

In [22]:
print(input_example_batch)
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

tf.Tensor(
[[26 57 47 ...  2  2  2]
 [20 58 50 ...  2  2  2]
 [23 55 44 ...  2  2  2]
 ...
 [22 56 45 ...  2  2  2]
 [20 58 57 ...  2  2  2]
 [24 61 58 ...  2  2  2]], shape=(64, 63), dtype=int64)
Input: 
 'Indigo Faerie##################################################'

Next Char Predictions: 
 "KioânXObd9Né,ûGvDoNà!é'VOVnâSF&UZt0CJxP oXû1'-,A'NAOIzqB'íVéVío"


In [23]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 63, 79)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.3706627


In [24]:
model.compile(optimizer='adam', loss=loss)

In [25]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [26]:
EPOCHS=10

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
319/319 [==============================] - 326s 1s/step - loss: 0.8386
Epoch 2/10
319/319 [==============================] - 326s 1s/step - loss: 0.5964
Epoch 3/10
319/319 [==============================] - 307s 962ms/step - loss: 0.5488
Epoch 4/10
319/319 [==============================] - 298s 935ms/step - loss: 0.5174
Epoch 5/10
 21/319 [>.............................] - ETA: 4:53 - loss: 0.5036

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 16

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"Visara"))